In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

#!mamba install -c conda-forge html5lib --yes
# import html5lib

import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


##### Scraping wiki url and create data frame

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, 'html.parser')

headings = ['Postal Code','Borough','Neighbourhood']


wiki_table = soup.find('table', class_= "wikitable sortable")

data = {'Postal Code':[], 'Borough':[],'Neighbourhood':[]}

rows = wiki_table.find_all('tr')

#print('number of rows: {}'.format(len(rows)))

row_marker = 0
for row in rows:
    column_marker = 0
    columns = row.find_all('td')
    for key, column in zip(data,columns):
        check = column.get_text()
        data[key].append(check[:-1])

df = pd.DataFrame(data)
df.head(11)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


##### Dropping rows with Borough 'Not assigned'

In [3]:
indexNames = df[df['Borough']== 'Not assigned'].index
indexNames
df_v2 = df.drop(indexNames).reset_index(drop=True)
df_v2.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### For rows with **Not assigned** Neighbourhood, adopt Borough value for Neighbourhood

In [4]:
indexNames2 = df_v2[df_v2['Neighbourhood']=='Not assigned'].index
df_v3 = df_v2

for ind in df_v2.index:
    if ind in indexNames2:
        df_v3.iloc[ind]['Neighbourhood']=df_v3.iloc[ind]['Borough']

df_v3.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


##### Checking shape of final dataframe

In [5]:
df_v3.shape

(103, 3)